In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

def lee_carter_projection_short(df_qx, projection_years):
    log_mx = np.log(df_qx.replace(0, np.nan)).dropna()
    ages = log_mx.index
    years = log_mx.columns.astype(int)
    ax = log_mx.mean(axis=1)
    centered = log_mx.sub(ax, axis=0)
    U, s, Vt = np.linalg.svd(centered, full_matrices=False)
    bx = U[:, 0]
    kt = s[0] * Vt[0, :]

    model = LinearRegression().fit(years.values.reshape(-1, 1), kt)
    future_years = list(range(years[-1] + 1, years[-1] + 1 + projection_years))
    kt_future = model.predict(np.array(future_years).reshape(-1, 1))

    log_mx_proj = pd.DataFrame({year: ax + bx * kt_val for year, kt_val in zip(future_years, kt_future)}, index=ages)
    full_log_mx = pd.concat([log_mx, log_mx_proj], axis=1)
    mx_proj = np.exp(full_log_mx)
    return mx_proj

def calcular_esperanza_vida_formato_mortalidad(df_mx, edades_interes):
    resultados = {edad: [] for edad in edades_interes}
    años = df_mx.columns

    for year in años:
        qx = 1 - np.exp(-df_mx[year])
        lx = [1.0]
        for q in qx.iloc[1:]:
            lx.append(lx[-1] * (1 - q))
        lx = np.array(lx)
        for edad in edades_interes:
            le = lx[edad:]
            ex = np.sum(le) / le[0]
            resultados[edad].append(ex)

    df_ex = pd.DataFrame(resultados, index=años).T
    df_ex.index.name = 'Edad'
    df_ex.columns.name = 'Año'
    return df_ex

# Parámetros
archivo = 'Tablas_Mortalidad_Proyectadas_Hombres.xlsx'  # Reemplaza con tu archivo real
projection_years = 2040 - 2017
edades_interes = [20, 30, 40, 50, 60, 70]

# Leer hojas
df_hombres = pd.read_excel(archivo, sheet_name='Hombres').set_index('edad')
df_mujeres = pd.read_excel(archivo, sheet_name='Mujeres').set_index('edad')

# Proyección de tasas mx
proj_hombres = lee_carter_projection_short(df_hombres, projection_years)
proj_mujeres = lee_carter_projection_short(df_mujeres, projection_years)

# Esperanza de vida con formato compatible
esperanza_hombres = calcular_esperanza_vida_formato_mortalidad(proj_hombres, edades_interes)
esperanza_mujeres = calcular_esperanza_vida_formato_mortalidad(proj_mujeres, edades_interes)
